In [1]:
import pandas as pd
import numpy as np
import datetime

## 要输入的地方是lotid，读取表格，如果有 CancelTrackIn 的要读两遍

In [88]:
L = 'L2E9228A2AA'
dfm = pd.read_excel('./%s/M_%s.xls' % (L, L))
dfcm = pd.read_excel('./%s/MC_%s.xls' % (L, L))
dfo = pd.read_excel('./%s/O_%s.xls' % (L, L))
print(dfm.shape)
print(dfcm.shape)
print(dfo.shape)

(702, 21)
(82, 21)
(827, 58)


## 这一步是找到有没有canceltrackin，如果有要删掉对应的trackin才行,如果执行了删除操作，就需要再读取一遍文件，也就是再run上一个cell，观察dfo的行数有没有变化

In [87]:
dfo[dfo['EventName'] == 'CancelTrackIn']

,Lot ID,CST ID,ProductQty,ProductSpec,ProcessFlowName,ProdType,Priority,EventTime,OLDOPER,NewOper,...,BeforeOperation,ReasonType,ReasonCode,ConsumerLotID,ConsumedLotID,ConsumedDurableName,ConsumedConsumableName,ProcessFlowVer,ProcessGrp,ACTUALSAMPLEPOSITION
193,L2E9228A2AA,AN0302,26,G2647FB101FA-007E,S1A576,E,3,2019-03-20 09:26:29.0,1A576,1A576,...,1A500,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
780,L2E9228A2AA,AN0302,26,G2647FB101FA-007E,S10176,E,3,2019-03-02 12:42:19.0,10176,10176,...,10170,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


## 全部的处理过程

In [89]:
Lotid = dfm['LOTID'][0]

dfcm['TIME'] = pd.to_datetime(dfcm['TIME'])
AUU_start = dfcm[(dfcm['DEST MACHINE'] == '2AUU01') & (dfcm['STATE'] == 'CREATED')]['TIME']
print('%s 进入Array时间为 %s 。' % (Lotid ,AUU_start.iloc[0]))

dfm = dfm[['TIME', 'TRANSPORTJOBID', 'STATE']]  #只选取我们需要用到的三列'TIME', 'TRANSPORTJOBID', 'STATE'
print(dfm.shape)
l = dfm['TRANSPORTJOBID']
dfmcount = pd.DataFrame([l.value_counts()]).T  # 统计每个TRANSPORTJOBID出现的次数
list_one = dfmcount[dfmcount['TRANSPORTJOBID'] == 1].index.tolist()  # 如果出现次数是1，记录下来下一步准备删除
print(len(list_one))
dfm=dfm[-dfm.TRANSPORTJOBID.isin(list_one)]  # 删除TRANSPORTJOBID是唯一的记录
print(dfm.shape)

dfm_time = pd.DataFrame()  # 新建一个表格用来处理搬送时间
# 原表中‘STATE’是‘CREATED’的那些行的时间提取出来
dfm_time['created'] = dfm[dfm['STATE'] == 'CREATED']['TIME'].tolist()
# 原表中‘STATE’不是‘CREATED’（也就是‘COMPLETED’或者是‘TERMINATED’）的那些行的时间提取出来
dfm_time['ended'] = dfm[dfm['STATE'] != 'CREATED']['TIME'].tolist()
# 转换成时间格式，为下一步计算差值以及总时间做准备
dfm_time = dfm_time.astype(np.datetime64)
# 计算每次搬送的时间，并求和
dfm_time['delta'] = dfm_time['ended'] - dfm_time['created']
trantime = dfm_time['delta'].sum()
print('%s 在 Array 搬送总时间为： %s' % (Lotid, trantime))

dfo = dfo[['EventTime', 'EventName', 'EQP ID']]                 # 只提取'EventTime', 'EventName'后续处理
dfo['EventTime'] = pd.to_datetime(dfo['EventTime'])   # 转换时间戳，便于后面的计算
shipidx = int(dfo[dfo['EventName'] == 'Ship'].index.values)
dfo = dfo.iloc[shipidx:]
ship_time = dfo[dfo['EventName'] == 'Ship']['EventTime']
sumtime = (ship_time.iloc[0] - AUU_start.iloc[0])
print('%s 在 Array 总的时间为：%s ' % (Lotid, sumtime))

dfo_time = pd.DataFrame()
dfo_time['tki'] = dfo[dfo['EventName'] == 'TrackIn']['EventTime'].tolist()
dfo_time['tko'] = dfo[dfo['EventName'] == 'TrackOut']['EventTime'].tolist()
dfo_time = dfo_time.astype(np.datetime64)
dfo_time['delta'] = dfo_time['tko'] - dfo_time['tki']
runtime = dfo_time['delta'].sum()
print('%s 在 Array 总的制程时间为 ： %s' % (Lotid, runtime))

dfo_time = pd.DataFrame()
dfo_time['tki'] = dfo[dfo['EventName'] == 'TrackIn']['EventTime'].tolist()
dfo_time['tko'] = dfo[dfo['EventName'] == 'TrackOut']['EventTime'].tolist()
dfo_time = dfo_time.astype(np.datetime64)
dfo_time['delta'] = dfo_time['tko'] - dfo_time['tki']
runtime = dfo_time['delta'].sum()
print('%s 在 Array 总的制程时间为 ： %s' % (Lotid, runtime))

waittime = (sumtime - trantime - runtime)
print('%s 在 Array 总的等待时间为：% s' % (Lotid, waittime))

L2E9228A2AA 进入Array时间为 2019-03-01 03:08:19.368000 。
(702, 3)
16
(686, 3)
L2E9228A2AA 在 Array 搬送总时间为： 1 days 10:34:50.770000
L2E9228A2AA 在 Array 总的时间为：35 days 19:05:09.632000 
L2E9228A2AA 在 Array 总的制程时间为 ： 7 days 03:50:56
L2E9228A2AA 在 Array 总的制程时间为 ： 7 days 03:50:56
L2E9228A2AA 在 Array 总的等待时间为：27 days 04:39:22.862000


## 下面格式输出，稍微美观一点

In [90]:
print('本批lot为： %s ' % Lotid)
print('%s 在 Array 总的时间为:       %s' % (Lotid, sumtime))
print('%s 在 Array 搬送总时间为:     %s' % (Lotid, trantime))
print('%s 在 Array 总的制程时间为:   %s' % (Lotid, runtime))
print('%s 在 Array 总的等待时间为:   %s' % (Lotid, waittime))

本批lot为： L2E9228A2AA 
L2E9228A2AA 在 Array 总的时间为:       35 days 19:05:09.632000
L2E9228A2AA 在 Array 搬送总时间为:     1 days 10:34:50.770000
L2E9228A2AA 在 Array 总的制程时间为:   7 days 03:50:56
L2E9228A2AA 在 Array 总的等待时间为:   27 days 04:39:22.862000


## 输出到csv

In [91]:
df_lot = pd.DataFrame()
df_lot['过程'] = ['%s 在 Array 总的时间为' % Lotid, '%s 在 Array 搬送总时间为' % Lotid, '%s 在 Array 总的制程时间为' % Lotid, '%s 在 Array 总的等待时间为' % Lotid]
df_lot['时间'] = [sumtime, trantime, runtime, waittime]

topath = './%s/res_%s.csv' % (L, L)
df_lot.to_csv(topath, encoding='utf_8_sig', index=False)
print('成功保存到res_%s.csv文件!' % Lotid)

成功保存到res_L2E9228A2AA.csv文件!


## 转换成秒，绘制饼图

In [ ]:
s_runtime = runtime.total_seconds()
s_trantime = trantime.total_seconds()
s_waittime = waittime.total_seconds()
# print(s_runtime, s_trantime, s_waittime)

from pyecharts import Pie

attr = ["制程时间", "搬送时间", "等待时间"]
s_time = [s_runtime, s_trantime, s_waittime]
pie = Pie(('%s在Array总时长%s' % (Lotid, sumtime)), width=900, height=500,  title_pos='center', title_text_size=21)
pie.add("", attr, s_time, 
        is_label_show=True,
        radius=[45, 70], 
        legend_pos = 'right',
        legend_orient = 'vertical',
        label_text_size=19, legend_text_size=17)

## 绘制在各个站点的runtime/ Notruntime

In [18]:
df = dfo
df = df[[ 'EventTime', 'EQP ID', 'EventName']]
ship_idx = int(df[df['EventName'] == 'Ship'].index.values)
df = df.iloc[ship_idx:]
df

,EventTime,EQP ID,EventName
33,2019-04-04 20:17:30.0,2ASS01,Ship
34,2019-04-04 19:07:59.0,2ASS01,ReleaseHold
35,2019-04-04 19:05:47.0,2ASS01,Hold
36,2019-04-04 19:03:05.0,2ASS01,TrackOut
37,2019-04-04 19:03:05.0,2ASS01,TransferProduct
38,2019-04-04 19:03:04.0,2ASS01,TrackIn
39,2019-04-04 19:02:46.0,2ATV01,LotInfoDownLoad
40,2019-04-04 18:48:34.0,2ATV01,"CreateSortJob, Split"
41,2019-04-04 18:47:38.0,2ATV01,ChangeFlowToSort
42,2019-04-04 18:47:15.0,2ATV01,ReleaseHold


In [19]:
df = df[df.EventName.isin(['TrackIn', 'TrackOut'])]
df = df.reset_index(drop=True)
df

,EventTime,EQP ID,EventName
0,2019-04-04 19:03:05.0,2ASS01,TrackOut
1,2019-04-04 19:03:04.0,2ASS01,TrackIn
2,2019-04-04 16:05:17.0,2ATV01,TrackOut
3,2019-04-04 15:43:55.0,2ATV01,TrackIn
4,2019-04-04 15:43:19.0,2ATO07,TrackOut
5,2019-04-04 15:00:59.0,2ATO07,TrackIn
6,2019-04-04 13:53:20.0,2AMM09,TrackOut
7,2019-04-04 12:39:58.0,2AMM09,TrackIn
8,2019-04-04 12:09:28.0,2AMS03,TrackOut
9,2019-04-04 11:14:54.0,2AMS03,TrackIn


In [ ]:
## 首先获取所有的index，并且

In [20]:
df_io = df.drop([0, int(len(df.index)-1)])
df_io

,EventTime,EQP ID,EventName
1,2019-04-04 19:03:04.0,2ASS01,TrackIn
2,2019-04-04 16:05:17.0,2ATV01,TrackOut
3,2019-04-04 15:43:55.0,2ATV01,TrackIn
4,2019-04-04 15:43:19.0,2ATO07,TrackOut
5,2019-04-04 15:00:59.0,2ATO07,TrackIn
6,2019-04-04 13:53:20.0,2AMM09,TrackOut
7,2019-04-04 12:39:58.0,2AMM09,TrackIn
8,2019-04-04 12:09:28.0,2AMS03,TrackOut
9,2019-04-04 11:14:54.0,2AMS03,TrackIn
10,2019-04-04 07:57:35.0,2AOV10,TrackOut
